In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.9 MB/s eta 0:00:00


# **Extraction des indicateurs_environnementaux**

In [11]:
import PyPDF2
import re
import csv

# Chargement du PDF
def load_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# Indicateurs et expressions associées
indicators = {
    "Émissions de CO₂": r"(émissions.*CO[₂2].*?(\d+[\.,]?\d*).*(tonnes?))",
    "Consommation d'énergie": r"(consommation.*énergie.*?(\d+[\.,]?\d*).*(kWh|%))",
    "Gestion des déchets": r"(déchets.*?(\d+[\.,]?\d*).*(tonnes?)|(recyclé.*?(\d+[\.,]?\d*).*(%)))",
    "Consommation d'eau": r"(consommation.*eau.*?(\d+[\.,]?\d*).*(m³))"
}

# Extraction des informations
def extract_info(text, indicators):
    results = []
    for indicator, pattern in indicators.items():
        matches = re.findall(pattern, text, re.IGNORECASE)
        for match in matches:
            value = "".join(filter(None, match))  # Filtrer les groupes vides
            results.append({
                "Catégorie": "Environnement",
                "Indicateur": indicator,
                "Valeur": value,
                "Contexte": match[0]  # La phrase complète contenant l'indicateur
            })
    return results

# Enregistrement dans un fichier CSV
def save_to_csv(data, file_name):
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Catégorie", "Indicateur", "Valeur", "Unité", "Contexte"])
        writer.writeheader()
        for row in data:
            writer.writerow(row)

# Chemin vers le fichier PDF
file_path = "/content/DPEF_4.pdf"

# Extraction des informations et sauvegarde
text = load_pdf(file_path)
data = extract_info(text, indicators)
save_to_csv(data, "resultats_indicateurs_environnementaux.csv")
print("Les résultats ont été enregistrés dans 'resultats_indicateurs_environnementaux.csv'.")


Les résultats ont été enregistrés dans 'resultats_indicateurs_environnementaux.csv'.


**Rapport DPEF_29**

In [12]:
import PyPDF2
import re
import csv

# Chargement du PDF
def load_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# Indicateurs et expressions associées
indicators = {
    "Émissions de CO₂": r"(émissions.*CO[₂2].*?(\d+[\.,]?\d*).*(tonnes?))",
    "Consommation d'énergie": r"(consommation.*énergie.*?(\d+[\.,]?\d*).*(kWh|%))",
    "Gestion des déchets": r"(déchets.*?(\d+[\.,]?\d*).*(tonnes?)|(recyclé.*?(\d+[\.,]?\d*).*(%)))",
    "Consommation d'eau": r"(consommation.*eau.*?(\d+[\.,]?\d*).*(m³))"
}

# Extraction des informations
def extract_info(text, indicators):
    results = []
    for indicator, pattern in indicators.items():
        matches = re.findall(pattern, text, re.IGNORECASE)
        for match in matches:
            value = "".join(filter(None, match))  # Filtrer les groupes vides
            results.append({
                "Catégorie": "Environnement",
                "Indicateur": indicator,
                "Valeur": value,
                "Contexte": match[0]  # La phrase complète contenant l'indicateur
            })
    return results

# Enregistrement dans un fichier CSV
def save_to_csv(data, file_name):
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Catégorie", "Indicateur", "Valeur", "Unité", "Contexte"])
        writer.writeheader()
        for row in data:
            writer.writerow(row)

# Chemin vers le fichier PDF
file_path = "/content/DPEF_29.pdf"

# Extraction des informations et sauvegarde
text = load_pdf(file_path)
data = extract_info(text, indicators)
save_to_csv(data, "resultats_indicateurs_environnementaux.csv")
print("Les résultats ont été enregistrés dans 'resultats_indicateurs_environnementeaux.csv'.")


Les résultats ont été enregistrés dans 'resultats_indicateurs_environnementeaux.csv'.


# **Extraction des indicateurs_sociaux**

In [14]:
import PyPDF2
import re
import csv

# Chargement et nettoyage du texte PDF
def load_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            # Utiliser extract_text() avec des options pour préserver la structure
            page_text = page.extract_text()
            text += page_text + "\n"  # Ajouter un saut de ligne entre les pages

    # Nettoyage plus avancé
    text = re.sub(r'\n+', '\n', text)  # Réduire les sauts de ligne multiples
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Fonction de nettoyage  du texte
def clean_text(text):
    # Supprimer les en-têtes, pieds de page, numéros de page
    text = re.sub(r'^.*?(?=\n)', '', text, flags=re.MULTILINE)
    text = re.sub(r'\n\d+\n', '', text)

    # Normaliser l'espace et la ponctuation
    text = re.sub(r'\s+', ' ', text)
    text = text.replace(' ,', ',').replace(' .', '.')

    return text.strip()

# Expressions régulières pour les indicateurs sociaux (avec critères affinés)
social_indicators = {
    "Conditions de travail et bien-être": {
        "pattern": r"(santé\s+au\s+travail|risques\s+professionnels|bien-être\s+employés|accidents\s+du\s+travail|conditions\s+de\s+travail)",
        "context_window": 150
    },
    "Diversité et inclusion": {
        "pattern": r"(politique\s+diversité|égalité\s+professionnelle|représentation\s+diversité|inclusion\s+professionnelle|égalité\s+des\s+chances)",
        "context_window": 150
    },
    "Formation et développement": {
        "pattern": r"(formation\s+professionnelle|développement\s+des\s+compétences|plan\s+de\s+formation|heures\s+de\s+formation|évolution\s+professionnelle)",
        "context_window": 150
    },
    "Relations avec les communautés": {
        "pattern": r"(engagement\s+communautaire|projets\s+sociaux|responsabilité\s+sociale|actions\s+solidaires|partenariats\s+locaux)",
        "context_window": 150
    },
    "Droits de l'homme et éthique": {
        "pattern": r"(droits\s+de\s+l'homme|éthique\s+professionnelle|respect\s+des\s+droits|politique\s+éthique|déontologie)",
        "context_window": 150
    }
}

# Extraction avec contexte
def extract_social_info(text, social_indicators, max_results=5):
    results = []
    for category, config in social_indicators.items():
        pattern = config["pattern"]
        context_window = config.get("context_window", 300)

        # Trouver toutes les correspondances avec leur contexte
        matches = list(re.finditer(pattern, text, re.IGNORECASE))

        for match in matches[:max_results]:
            # Extraire le contexte autour de la correspondance
            start = max(0, match.start() - context_window)
            end = min(len(text), match.end() + context_window)

            context = text[start:end].strip()

            results.append({
                "Catégorie": category,
                "Mot-clé": match.group(0),
                "Contexte": context
            })

            if len(results) >= max_results:
                break

    return results

# Enregistrement des résultats dans un fichier CSV
def save_to_csv(data, file_name):
    with open(file_name, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=["Catégorie", "Mot-clé", "Contexte"])
        writer.writeheader()
        for row in data:
            writer.writerow(row)

# Fonction principale
def main(file_path):
    try:
        # Charger et nettoyer le texte
        text = load_pdf(file_path)
        text = clean_text(text)

        # Extraire les informations sociales
        social_data = extract_social_info(text, social_indicators)

        # Sauvegarder les résultats
        save_to_csv(social_data, "resultats_sociaux.csv")

        print("Extraction terminée avec succès. Résultats enregistrés dans 'resultats_sociaux.csv'.")

    except Exception as e:
        print(f"Une erreur est survenue : {e}")

# Chemin vers le fichier PDF
file_path = "/content/DPEF_29.pdf"

# Exécution du script
if __name__ == "__main__":
    main(file_path)

Extraction terminée avec succès. Résultats enregistrés dans 'resultats_sociaux.csv'.


# **Extraction des indicateurs_de gouvernance**

In [15]:
import PyPDF2
import re
import csv

def load_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            text += page_text + "\n"

    # Nettoyage du texte
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def clean_text(text):
    # Nettoyage approfondi du texte
    text = re.sub(r'^.*?(?=\n)', '', text, flags=re.MULTILINE)
    text = re.sub(r'\n\d+\n', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.replace(' ,', ',').replace(' .', '.')
    return text.strip()

# Définition des indicateurs de gouvernance avec des patterns plus précis
governance_indicators = {
    "Structure du conseil d'administration": {
        "pattern": r"(conseil\s+d'administration|composition\s+du\s+conseil|indépendance\s+des\s+administrateurs|diversité\s+des\s+membres|compétences\s+du\s+conseil)",
        "context_window": 250
    },
    "Éthique et intégrité des affaires": {
        "pattern": r"(politique\s+anti-corruption|conformité\s+légale|prévention\s+des\s+fraudes|programme\s+de\s+dénonciation|éthique\s+des\s+affaires)",
        "context_window": 250
    },
    "Rémunération des dirigeants": {
        "pattern": r"(rémunération\s+des\s+dirigeants|transparence\s+salariale|rémunération\s+variable|lien\s+performance-salaire|politique\s+de\s+rémunération)",
        "context_window": 250
    },
    "Gestion des risques": {
        "pattern": r"(gestion\s+des\s+risques|risques\s+environnementaux|risques\s+sociaux|risques\s+de\s+gouvernance|processus\s+d'évaluation\s+des\s+risques)",
        "context_window": 250
    },
    "Transparence et communication": {
        "pattern": r"(rapport\s+ESG|publication\s+d'informations|engagement\s+des\s+parties\s+prenantes|communication\s+financière|transparence\s+corporate)",
        "context_window": 250
    }
}

def extract_governance_info(text, governance_indicators, max_results=3):
    results = []
    for category, config in governance_indicators.items():
        pattern = config["pattern"]
        context_window = config.get("context_window", 250)

        # Trouver toutes les correspondances avec leur contexte
        matches = list(re.finditer(pattern, text, re.IGNORECASE))

        for match in matches[:max_results]:
            # Extraire le contexte autour de la correspondance
            start = max(0, match.start() - context_window)
            end = min(len(text), match.end() + context_window)

            context = text[start:end].strip()

            results.append({
                "Catégorie": category,
                "Mot-clé": match.group(0),
                "Contexte": context
            })

            if len(results) >= max_results:
                break

    return results

def save_to_csv(data, file_name):
    with open(file_name, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=["Catégorie", "Mot-clé", "Contexte"])
        writer.writeheader()
        for row in data:
            writer.writerow(row)

def main(file_path):
    try:
        # Charger et nettoyer le texte
        text = load_pdf(file_path)
        text = clean_text(text)

        # Extraire les informations de gouvernance
        governance_data = extract_governance_info(text, governance_indicators)

        # Sauvegarder les résultats
        save_to_csv(governance_data, "resultats_gouvernance.csv")

        print("Extraction de la gouvernance terminée avec succès. Résultats enregistrés dans 'resultats_gouvernance.csv'.")

    except Exception as e:
        print(f"Une erreur est survenue : {e}")

# Chemin vers le fichier PDF
file_path = "/content/DPEF_29.pdf"

# Exécution du script
if __name__ == "__main__":
    main(file_path)

Extraction de la gouvernance terminée avec succès. Résultats enregistrés dans 'resultats_gouvernance.csv'.


In [16]:
import PyPDF2
import re
import csv

def load_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            text += page_text + "\n"

    # Nettoyage du texte
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def clean_text(text):
    # Nettoyage approfondi du texte
    text = re.sub(r'^.*?(?=\n)', '', text, flags=re.MULTILINE)
    text = re.sub(r'\n\d+\n', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.replace(' ,', ',').replace(' .', '.')
    return text.strip()

# Définition des indicateurs de gouvernance avec des patterns plus précis
governance_indicators = {
    "Structure du conseil d'administration": {
        "pattern": r"(conseil\s+d'administration|composition\s+du\s+conseil|indépendance\s+des\s+administrateurs|diversité\s+des\s+membres|compétences\s+du\s+conseil)",
        "context_window": 250
    },
    "Éthique et intégrité des affaires": {
        "pattern": r"(politique\s+anti-corruption|conformité\s+légale|prévention\s+des\s+fraudes|programme\s+de\s+dénonciation|éthique\s+des\s+affaires)",
        "context_window": 250
    },
    "Rémunération des dirigeants": {
        "pattern": r"(rémunération\s+des\s+dirigeants|transparence\s+salariale|rémunération\s+variable|lien\s+performance-salaire|politique\s+de\s+rémunération)",
        "context_window": 250
    },
    "Gestion des risques": {
        "pattern": r"(gestion\s+des\s+risques|risques\s+environnementaux|risques\s+sociaux|risques\s+de\s+gouvernance|processus\s+d'évaluation\s+des\s+risques)",
        "context_window": 250
    },
    "Transparence et communication": {
        "pattern": r"(rapport\s+ESG|publication\s+d'informations|engagement\s+des\s+parties\s+prenantes|communication\s+financière|transparence\s+corporate)",
        "context_window": 250
    }
}

def extract_governance_info(text, governance_indicators, max_results=3):
    results = []
    for category, config in governance_indicators.items():
        pattern = config["pattern"]
        context_window = config.get("context_window", 250)

        # Trouver toutes les correspondances avec leur contexte
        matches = list(re.finditer(pattern, text, re.IGNORECASE))

        for match in matches[:max_results]:
            # Extraire le contexte autour de la correspondance
            start = max(0, match.start() - context_window)
            end = min(len(text), match.end() + context_window)

            context = text[start:end].strip()

            results.append({
                "Catégorie": category,
                "Mot-clé": match.group(0),
                "Contexte": context
            })

            if len(results) >= max_results:
                break

    return results

def save_to_csv(data, file_name):
    with open(file_name, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=["Catégorie", "Mot-clé", "Contexte"])
        writer.writeheader()
        for row in data:
            writer.writerow(row)

def main(file_path):
    try:
        # Charger et nettoyer le texte
        text = load_pdf(file_path)
        text = clean_text(text)

        # Extraire les informations de gouvernance
        governance_data = extract_governance_info(text, governance_indicators)

        # Sauvegarder les résultats
        save_to_csv(governance_data, "resultats_gouvernance.csv")

        print("Extraction de la gouvernance terminée avec succès. Résultats enregistrés dans 'resultats_gouvernance.csv'.")

    except Exception as e:
        print(f"Une erreur est survenue : {e}")

# Chemin vers le fichier PDF
file_path = "/content/DPEF_4.pdf"

# Exécution du script
if __name__ == "__main__":
    main(file_path)

Extraction de la gouvernance terminée avec succès. Résultats enregistrés dans 'resultats_gouvernance.csv'.
